In [28]:
from __future__ import print_function

import numpy as np
import pandas as pd

from os import getcwd
from os.path import join, basename

from utils.literature import DataLoader
from glob import glob

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 

nltk.download()

In [3]:
cwd = getcwd()

json_paths = [
    join(cwd, 'dataset', 'arxiv', 'arxiv', 'pdf_json'),
    join(cwd, 'dataset', 'arxiv', 'arxiv', 'pdf_json'),
    join(cwd, 'dataset', 'comm_use_subset', 'comm_use_subset', 'pdf_json'),
    join(cwd, 'dataset', 'noncomm_use_subset', 'noncomm_use_subset', 'pdf_json'),
    join(cwd, 'dataset', 'custom_license', 'custom_license', 'pdf_json'),
    join(cwd, 'dataset' 'biorxiv_medrxiv', 'biorxiv_medrxiv', 'pdf_json'),
]

files = []
[files.extend(glob(join(path, '*.json'))) for path in json_paths];

In [4]:
metadata = pd.read_csv(join(getcwd(), 'dataset', 'metadata.csv'))

In [5]:
metadata['doc_id'] = np.arange(len(metadata))

In [6]:
metadata.head(2)

,cord_uid,sha,source_x,title,doi,pmcid,pubmed_id,license,abstract,publish_time,authors,journal,Microsoft Academic Paper ID,WHO #Covidence,arxiv_id,has_pdf_parse,has_pmc_xml_parse,full_text_file,url,doc_id
0,zjufx4fo,b2897e1277f56641193a6db73825f707eed3e4c9,PMC,Sequence requirements for RNA strand transfer ...,10.1093/emboj/20.24.7220,PMC125340,11742998.0,unk,Nidovirus subgenomic mRNAs contain a leader se...,2001-12-17,"Pasternak, Alexander O.; van den Born, Erwin; ...",The EMBO Journal,NaN,NaN,NaN,True,True,custom_license,http://europepmc.org/articles/pmc125340?pdf=re...,0
1,ymceytj3,e3d0d482ebd9a8ba81c254cc433f314142e72174,PMC,"Crystal structure of murine sCEACAM1a[1,4]: a ...",10.1093/emboj/21.9.2076,PMC125375,11980704.0,unk,CEACAM1 is a member of the carcinoembryonic an...,2002-05-01,"Tan, Kemin; Zelus, Bruce D.; Meijers, Rob; Liu...",The EMBO Journal,NaN,NaN,NaN,True,True,custom_license,http://europepmc.org/articles/pmc125375?pdf=re...,1


In [21]:
doc_idxs = dict(zip(metadata['doc_id'], metadata['title']))
doc_idxs[100]

'Information theory-based algorithm for in silico prediction of PCR products with whole genomic sequences as templates'

In [22]:
stop_words = set(stopwords.words('english'))

In [23]:
def get_metadata(metadata, fpath):
    sha = basename(fpath[0])[:-5]
    return metadata.loc[metadata['sha'] == sha]

In [24]:
def get_full_text(fpath):
    dl = DataLoader(fpath)
    return dl.get_full_text()

In [25]:
word_idxs = dict()

In [29]:
for i, fpath in enumerate(files):
    print (f"{i} of {len(files)} files processed", end="\r")

    #get the corresponding metadata
    doc_meta = get_metadata(metadata, fpath)
    doc_id = doc_meta['doc_id']
    
    #preprocessing
    tokens = word_tokenize(get_full_text(fpath))
    filtered_tokens = [w for w in tokens if not w in stop_words] 

    for token in filtered_tokens:
        if token not in word_idxs.keys():
            word_idxs[token] = len(word_idxs)

46526 of 4652846527 of 46528

In [32]:
len(word_idxs)

2668371

In [31]:
word_idxs['sars-cov-2']

3983